In [1]:
%matplotlib inline
import numpy as np
import pandas as pd
import scipy
import sklearn
import spacy
import matplotlib.pyplot as plt
import seaborn as sns
import re
from nltk.corpus import gutenberg, stopwords
from collections import Counter
import nltk

nltk.download('gutenberg')
!python -m spacy download en

[nltk_data] Downloading package gutenberg to
[nltk_data]     C:\Users\wscott\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\gutenberg.zip.


[+] Download and installation successful
You can now load the model via spacy.load('en_core_web_sm')
symbolic link created for C:\Users\wscott\AppData\Local\Programs\Python\Python37\lib\site-packages\spacy\data\en <<===>> C:\Users\wscott\AppData\Local\Programs\Python\Python37\lib\site-packages\en_core_web_sm
[+] Linking successful
C:\Users\wscott\AppData\Local\Programs\Python\Python37\lib\site-packages\en_core_web_sm
-->
C:\Users\wscott\AppData\Local\Programs\Python\Python37\lib\site-packages\spacy\data\en
You can now load the model via spacy.load('en')


You should consider upgrading via the 'python -m pip install --upgrade pip' command.


In [2]:
# Utility function for standard text cleaning.
def text_cleaner(text):
    # Visual inspection identifies a form of punctuation spaCy does not
    # recognize: the double dash '--'.  Better get rid of it now!
    text = re.sub(r'--',' ',text)
    text = re.sub("[\[].*?[\]]", "", text)
    text = ' '.join(text.split())
    return text
    
# Load and clean the data.
persuasion = gutenberg.raw('austen-persuasion.txt')
alice = gutenberg.raw('carroll-alice.txt')

# The Chapter indicator is idiosyncratic
persuasion = re.sub(r'Chapter \d+', '', persuasion)
alice = re.sub(r'CHAPTER .*', '', alice)
    
alice = text_cleaner(alice[:int(len(alice)/10)])
persuasion = text_cleaner(persuasion[:int(len(persuasion)/10)])

In [3]:
# Parse the cleaned novels. This can take a bit.
nlp = spacy.load('en')
alice_doc = nlp(alice)
persuasion_doc = nlp(persuasion)

In [5]:
# Group into sentences.
alice_sents = [[sent, "Carroll"] for sent in alice_doc.sents]
persuasion_sents = [[sent, "Austen"] for sent in persuasion_doc.sents]

# Combine the sentences from the two novels into one data frame.
sentences = pd.DataFrame(alice_sents + persuasion_sents)
sentences.head()

,0,1
0,"(Alice, was, beginning, to, get, very, tired, ...",Carroll
1,"(So, she, was, considering, in, her, own, mind...",Carroll
2,"(There, was, nothing, so, VERY, remarkable, in...",Carroll
3,"(Oh, dear, !)",Carroll
4,"(Oh, dear, !)",Carroll


In [6]:
# Utility function to create a list of the 2000 most common words.
def bag_of_words(text):
    
    # Filter out punctuation and stop words.
    allwords = [token.lemma_
                for token in text
                if not token.is_punct
                and not token.is_stop]
    
    # Return the most common words.
    return [item[0] for item in Counter(allwords).most_common(2000)]
    

# Creates a data frame with features for each word in our common word set.
# Each value is the count of the times the word appears in each sentence.
def bow_features(sentences, common_words):
    
    # Scaffold the data frame and initialize counts to zero.
    df = pd.DataFrame(columns=common_words)
    df['text_sentence'] = sentences[0]
    df['text_source'] = sentences[1]
    df.loc[:, common_words] = 0
    
    # Process each row, counting the occurrence of words in each sentence.
    for i, sentence in enumerate(df['text_sentence']):
        
        # Convert the sentence to lemmas, then filter out punctuation,
        # stop words, and uncommon words.
        words = [token.lemma_
                 for token in sentence
                 if (
                     not token.is_punct
                     and not token.is_stop
                     and token.lemma_ in common_words
                 )]
        
        # Populate the row with word counts.
        for word in words:
            df.loc[i, word] += 1
        
        # This counter is just to make sure the kernel didn't hang.
        if i % 50 == 0:
            print("Processing row {}".format(i))
            
    return df

# Set up the bags.
alicewords = bag_of_words(alice_doc)
persuasionwords = bag_of_words(persuasion_doc)

# Combine bags to create a set of unique words.
common_words = set(alicewords + persuasionwords)

In [7]:
# Create our data frame with features. This can take a while to run.
word_counts = bow_features(sentences, common_words)
word_counts.head()

Processing row 0
Processing row 50
Processing row 100
Processing row 150
Processing row 200
Processing row 250
Processing row 300
Processing row 350
Processing row 400


,Morley,opposite,Latitude,name,printer,second,sir,heaven,final,bad,...,correspondent,home,worth,impose,trot,hedge,save,20,text_sentence,text_source
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,"(Alice, was, beginning, to, get, very, tired, ...",Carroll
1,0,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,"(So, she, was, considering, in, her, own, mind...",Carroll
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,"(There, was, nothing, so, VERY, remarkable, in...",Carroll
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,"(Oh, dear, !)",Carroll
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,"(Oh, dear, !)",Carroll


In [9]:
from sklearn import ensemble
from sklearn.model_selection import train_test_split

rfc = ensemble.RandomForestClassifier()
Y = word_counts['text_source']
X = np.array(word_counts.drop(['text_sentence','text_source'], 1))

X_train, X_test, y_train, y_test = train_test_split(X, 
                                                    Y,
                                                    test_size=0.4,
                                                    random_state=0)
train = rfc.fit(X_train, y_train)

print('Training set score:', rfc.score(X_train, y_train))
print('\nTest set score:', rfc.score(X_test, y_test))

c:\users\wscott\appdata\local\programs\python\python37\lib\site-packages\sklearn\ensemble\forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


Training set score: 0.9887218045112782

Test set score: 0.8370786516853933


In [13]:
y_train[:5]

440     Austen
116    Carroll
168     Austen
368     Austen
120    Carroll
Name: text_source, dtype: object

In [23]:
y_train_svm = y_train.replace('Austen', 1)
y_train_svm = y_train_svm.replace('Carroll', 0)

y_test_svm = y_test.replace('Austen', 1)
y_test_svm = y_test_svm.replace('Carroll', 0)

In [24]:
y_train_svm[:5]

440    1
116    0
168    1
368    1
120    0
Name: text_source, dtype: int64

In [29]:
from sklearn.svm import SVC
svc = SVC(probability=True)
svc.fit(X_train,y_train_svm)

c:\users\wscott\appdata\local\programs\python\python37\lib\site-packages\sklearn\svm\base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)


SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='auto_deprecated',
    kernel='rbf', max_iter=-1, probability=True, random_state=None,
    shrinking=True, tol=0.001, verbose=False)

In [30]:
print('Training set score:', svc.score(X_train, y_train_svm))
print('\nTest set score:', svc.score(X_test, y_test_svm))

Training set score: 0.7142857142857143

Test set score: 0.702247191011236


In [89]:
def predict_threshhold(prob_array):
    predict_array = []
    for t in prob_array:
        if t[1] > .5:
            predict_array.append(1)
        else:
            predict_array.append(0)
    return predict_array

In [90]:
y_threshhold_pred = predict_threshhold(predict_threshold)

In [91]:
y_threshhold_pred[:5]

[1, 1, 1, 0, 0]

In [93]:
from sklearn.metrics import classification_report,accuracy_score,confusion_matrix,auc,roc_auc_score,precision_score,recall_score

In [94]:
print('\nConfusion Matrix: \n')
print (confusion_matrix(y_test_svm,y_threshhold_pred))


Confusion Matrix: 

[[ 37  16]
 [  9 116]]


In [99]:
svc_pred = svc.predict(X_test)

In [100]:
print('\nConfusion Matrix: \n')
print (confusion_matrix(y_test_svm,svc_pred))


Confusion Matrix: 

[[  0  53]
 [  0 125]]


In [114]:
len(np.where(y_train == "Austen"))

1

In [118]:
new = np.where(y_train == "Austen")

In [132]:
np.transpose(new).shape

(190, 1)

In [117]:
y_train.shape

(266,)

In [103]:
y_train["Austen"]

KeyError: 'Austen'

In [39]:
X_test[:1].shape

(1, 1612)

In [44]:
svc.predict_proba(X_test[:4])

array([[0.07899331, 0.92100669],
       [0.08630354, 0.91369646],
       [0.12557073, 0.87442927],
       [0.56993291, 0.43006709]])

In [45]:
y_test_svm[:4]

327    1
233    1
122    0
102    0
Name: text_source, dtype: int64

In [46]:
svc.score(X_test[:4], y_test_svm[:4])

0.5

In [47]:
svc.predict(X_test[:4])

array([1, 1, 1, 1], dtype=int64)

In [48]:
test_prob = svc.predict_proba(X_test[:4])

In [58]:
test_prob[:,1]

array([0.92100669, 0.91369646, 0.87442927, 0.43006709])

In [64]:
predict_threshold = svc.predict_proba(X_test)

In [66]:
predict_threshold[:5,1]

array([9.21006689e-01, 9.13696462e-01, 8.74429268e-01, 4.30067089e-01,
       8.31593512e-06])

In [70]:
go = predict_threshold[:1]

In [73]:
go

array([[0.07899331, 0.92100669]])

In [74]:
go[:,1]

array([0.92100669])

In [81]:
test_prob[:,1]

array([[0.07899331, 0.92100669],
       [0.08630354, 0.91369646],
       [0.12557073, 0.87442927],
       [0.56993291, 0.43006709]])

In [84]:
predict_array = {}
for t in test_prob:
    print(t)

[0.07899331 0.92100669]
[0.08630354 0.91369646]
[0.12557073 0.87442927]
[0.56993291 0.43006709]


In [86]:
predict_array = []
for t in test_prob:
    if t[1] > .5:
        predict_array.append(1)
    else:
        predict_array.append(0)

In [87]:
predict_array

[1, 1, 1, 0]

In [ ]:
cross_val_score(svr, X, Y, cv=5)